In [ ]:
import tweepy
from tweepy import OAuthHandler
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import re


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

consumer_key = Con_key
consumer_secret = Con_secret
access_token = Access_token
access_token_secret = Token_secret

def authenticate_twitter():
    try:
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth)
        return api
    except Exception as e:
        print("Error during authentication:", e)
        return None

def fetch_tweets(api, query, count=100):
    try:
        tweets = []
        for tweet in tweepy.Cursor(api.search_tweets, q=query, lang='en', tweet_mode='extended').items(count):
            tweets.append(tweet.full_text)
        return tweets
    except Exception as e:
        print("Error fetching tweets:", e)
        return []

def clean_tweet(tweet):
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'\@\w+|\#', '', tweet)
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)
    tweet = tweet.lower()
    return tweet

def preprocess_tweet(tweet):
    tokens = word_tokenize(tweet)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

def analyze_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(tweet)
    if scores['compound'] >= 0.05:
        return 'Positive', scores['compound']
    elif scores['compound'] <= -0.05:
        return 'Negative', scores['compound']
    else:
        return 'Neutral', scores['compound']

def main():
    api = authenticate_twitter()
    if not api:
        return

    query = input("Enter a search term to analyze tweets: ")
    num_tweets = int(input("How many tweets to analyze? (max 100): "))

    raw_tweets = fetch_tweets(api, query, min(num_tweets, 100))
    if not raw_tweets:
        print("No tweets found or error occurred.")
        return

    results = []
    for tweet in raw_tweets:
        cleaned = clean_tweet(tweet)
        processed = preprocess_tweet(cleaned)
        sentiment, score = analyze_sentiment(processed)
        results.append({
            'Original Tweet': tweet,
            'Cleaned Tweet': cleaned,
            'Processed Text': processed,
            'Sentiment': sentiment,
            'Sentiment Score': score
        })

    df = pd.DataFrame(results)
    print("\nSentiment Analysis Results:")
    print(df[['Original Tweet', 'Sentiment', 'Sentiment Score']].head())

    print("\nSentiment Distribution:")
    print(df['Sentiment'].value_counts())

if __name__ == "__main__":
    main()